<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla, Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias, Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Martín Reyes
- Nombre de alumno 2: Lennin CHamorro


### **Link de repositorio de GitHub:** `https://github.com/Landilus/Lab_prog_cientifica`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [5]:
df_retail = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/LPC/Datos_Lab7/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [6]:
def custom_features(dataframe_in):
    #Se crea una copia del dataframe para manipularlo
    copy = dataframe_in.copy()
    #se procede a hacer cuatro agregaciones distintas, para manejar las columnas solicitadas de manera más seccionada
    #se crea la tabla Monetary junto con el ID del consumidor para hacer merge proximamente
    copy['Monetary'] = copy['Quantity'] * copy['Price']#se calcula cuanto se gasto en cada registro del dataframe de entrada
    total_compra = copy.groupby(['Invoice'])['Invoice', 'Monetary'].agg({'Monetary': 'sum'}).reset_index()#se agrupa el dinero gastado por boleta
    referencia = copy[~copy['Invoice'].duplicated()][['Invoice', 'Customer ID']]#se asocia la boleta con su respectivo consumidor
    pre_monetary = pd.merge(total_compra, referencia, on='Invoice')#se unen referencia y total_compra para hacer un dataframe que pueda calcular Monetary para cada ID
    Monetary = pre_monetary.groupby('Customer ID')['Monetary'].mean().reset_index()#se calcula el promedio de gasto por boleta de cada consumidor
    
    #se crea la tabla Periodicity junto con el ID del consumidor para hacer merge proximamente
    fecha_boleta = copy.groupby(['Invoice'])[ 'Invoice', 'InvoiceDate'].max().reset_index()#Se obtiene una tabla con la fecha de cada boleta
    Dif = pd.merge(fecha_boleta,referencia, on = 'Invoice')#se unen referencia y fecha_boleta para hacer un dataframe que pueda calcular Periodicity para cada ID
    Dif['Periodicity'] = Dif.groupby('Customer ID')['InvoiceDate'].diff().dropna().dt.days.astype(int)#Se crea una columna que contrnga la resta entrela fecha de la boleta y la siguiente del mismo consumidor
    Dif = Dif.fillna(0)# se rellenan con 0 los casos en que solo haya un registro por consumidor y no se pueda calcular la diferencia
    Periodicity = Dif.groupby('Customer ID')['Periodicity'].std().reset_index()#se calcula la desviacion estandar de las diferencias agrupando por cada consumidor
    Periodicity = Periodicity.fillna(0)#se vuelve a rellenar con 0 los casos que por ser un solo registro no se calcule la desviacion estandar de buena manera
    
    #se crea la tabla Frequency junto con el ID del consumidor para hacer merge proximamente
    Frequency = Dif.fillna(0).groupby('Customer ID')['Invoice'].count().reset_index()#se aprovecha la tabla Dif creada anteriormente para contar la cantidad de boletas por consumidor
    Frequency = Frequency.rename(columns = {'Invoice': 'Frequency'})#Se cambia el nombre para mayor facilidad al hacer merge

    #se crea la tabla que contiene Length y Recency junto con el ID del consumidor para hacer merge proximamente
    #Se crean las funciones necesarias para crear Length y Recency
    def Length(x):
        return (x.max() - x.min()).days
    def Recency(x):
        from datetime import datetime
        now = datetime.now()
        return (now - x.max()).days
    #Se realiza la agregacion Con las columnas Length y Recency
    LRMFP = copy.groupby('Customer ID').agg({'InvoiceDate' : [Length, Recency]}).reset_index()#Se hace la agrupacion aplicando la funcion Length y Recency agrupando por los ID de consumidor
    #Aqui hubo un problema con los niveles de la agregacion que no se dejaban manipular porque decia que era solo uno por lo que no se pudo utilizar droplevel
    LRMFP = pd.concat([LRMFP[i] for i in set([tupla[0] for tupla in LRMFP.columns])], axis = 1)


    #Se hace merge sobre todas las tablas creadas respecto al ID del consumidor y se ordena para retornar
    LRMFP = pd.merge(Frequency, pd.merge(Periodicity, pd.merge(LRMFP, Monetary, on='Customer ID'), on = 'Customer ID'), on = 'Customer ID')
    orden = ['Customer ID', 'Length', 'Recency', 'Frequency', 'Monetary', 'Periodicity']
    LRMFP = LRMFP.reindex(columns = orden)
    
    return LRMFP


**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [7]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
         return self
    def transform(self, X):
        return (X - np.min(X))/(np.max(X) - np.min(X))

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [8]:
LRMFP_transform = FunctionTransformer(func = custom_features)

MinMax_transform = ColumnTransformer([
    #('ignorar', 'passthrough', ['Customer ID']),
    ('MinMax', MinMax(), ['Length', 'Recency','Frequency', 'Monetary', 'Periodicity'])
    ])
tsne = TSNE(n_components=2, random_state=23)

In [9]:
pipeline = Pipeline([
    ('Obtencion Datos', LRMFP_transform),  # Paso de obtencion de datos: Se convierte df_retail en LRMFP
    ('preprocesamiento', MinMax_transform),  # Paso de preprocesamiento: Se le aplica MinMax Scaler
    ('Reduccion Dimensionalidad', tsne)  # Paso de reduccion de dimensionalidad: Se le aplica T-SNE
])



In [10]:
pipe_results = pipeline.fit_transform(df_retail)

<ipython-input-6-4094e8d95580>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  total_compra = copy.groupby(['Invoice'])['Invoice', 'Monetary'].agg({'Monetary': 'sum'}).reset_index()#se agrupa el dinero gastado por boleta
<ipython-input-6-4094e8d95580>:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  fecha_boleta = copy.groupby(['Invoice'])[ 'Invoice', 'InvoiceDate'].max().reset_index()#Se obtiene una tabla con la fecha de cada boleta
<ipython-input-6-4094e8d95580>:13: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  fecha_boleta = copy.groupby(['Invoice'])[ 'Invoice', 'InvoiceDate'].max().reset_index()#Se obtiene una tabla con la fecha de cada boleta
/usr/local/lib/

In [11]:
pipe_results_x = [tupla[0] for tupla in pipe_results]
pipe_results_y = [tupla[1] for tupla in pipe_results]

In [12]:
px.scatter(x = pipe_results_x, y = pipe_results_y)

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [13]:
pipeline_codo = Pipeline([
    ('Obtencion Datos', LRMFP_transform),  # Paso de obtencion de datos: Se convierte df_retail en LRMFP
    ('preprocesamiento', MinMax_transform),  # Paso de preprocesamiento: Se le aplica MinMax Scaler
    #('Reduccion Dimensionalidad', tsne)  # Paso de reduccion de dimensionalidad: Se le aplica T-SNE
])



In [14]:
df_transformado_codo = pipeline_codo.fit_transform(df_retail)

<ipython-input-6-4094e8d95580>:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-6-4094e8d95580>:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-6-4094e8d95580>:13: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning:

In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning:

In a future version, DataFrame.max(axis=None) will return a scalar max over the entire

In [15]:
intertias = [
    [i, KMeans(n_clusters=i, random_state=0).fit(df_transformado_codo).inertia_]
    for i in range(2, 20)
]

intertias = pd.DataFrame(intertias, columns=["n° clusters", "inertia"])
intertias.head(10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

,n° clusters,inertia
0,2,427.570740
1,3,266.096999
2,4,196.627487
3,5,154.013147
4,6,133.050741
5,7,114.617182
6,8,99.923112
7,9,90.072737
8,10,81.394393
9,11,75.438578


In [16]:
px.line(
    intertias,
    x="n° clusters",
    y="inertia",
    title="Método del Codo con K-Means",
    height=600,
)

Observando el gráfico y los valores de la inercia decidimos que el K ideal debiera encontrarse en 6 clusters.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [51]:
LRMFP = custom_features(df_retail)

<ipython-input-6-4094e8d95580>:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-6-4094e8d95580>:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-6-4094e8d95580>:13: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.



In [52]:
pipeline_k_means = Pipeline([
    ('Obtencion Datos', LRMFP_transform),  # Paso de obtencion de datos: Se convierte df_retail en LRMFP
    ('preprocesamiento', MinMax_transform),  # Paso de preprocesamiento: Se le aplica MinMax Scaler
    ('Clustering', KMeans(n_clusters=6, random_state=147, n_init=20)) # Paso de clustering: Se aplica k_means 
    #('Reduccion Dimensionalidad', tsne)  # Paso de reduccion de dimensionalidad: Se le aplica T-SNE
])

In [53]:
df_results_clus = pipeline_k_means.fit(df_retail)['Clustering'].labels_

<ipython-input-6-4094e8d95580>:7: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-6-4094e8d95580>:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-6-4094e8d95580>:13: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning:

In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning:

In a future version, DataFrame.max(axis=None) will return a scalar max over the entire

In [54]:
LRMFP['Cluster'] = df_results_clus

In [55]:
LRMFP

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity,Cluster
0,12346.0,196,4720,11,33.896364,35.257237,5
1,12347.0,37,4558,2,661.660000,26.162951,0
2,12348.0,0,4629,1,222.160000,0.000000,0
3,12349.0,181,4598,3,890.380000,88.791892,5
4,12351.0,0,4566,1,300.930000,0.000000,0
...,...,...,...,...,...,...,...
4309,18283.0,275,4573,6,103.228333,59.628014,1
4310,18284.0,0,4622,1,461.680000,0.000000,0
4311,18285.0,0,4851,1,427.000000,0.000000,2
4312,18286.0,247,4667,2,648.215000,174.655375,4


In [56]:
Clustering_stats = LRMFP.groupby(['Cluster']).agg({'Length': 'mean', 'Recency': 'mean', 'Frequency': 'mean', 'Monetary': 'mean', 'Periodicity': 'mean', 'Customer ID': 'count'}).reset_index()
Clustering_stats = Clustering_stats.rename(columns={'Customer ID': 'N_Registros'})
Clustering_stats

,Cluster,Length,Recency,Frequency,Monetary,Periodicity,N_Registros
0,0,16.742609,4598.511304,1.772174,379.686824,7.617074,1150
1,1,320.641246,4576.482277,11.795918,424.997019,40.822203,931
2,2,5.170354,4862.019912,1.263274,353.505732,2.771551,452
3,3,27.419130,4745.500870,1.760000,341.781213,13.462181,575
4,4,274.180064,4599.900322,2.717042,356.622958,154.012197,311
5,5,175.036872,4623.631285,4.208939,361.331110,60.638138,895


Observando los Datos podemos ver que si hay diferencias importantes entre la mayoría de los clusters, con esto podemos darles atributos a los clientes, dependiendo de su lealtad, aporte monetario al negocio y si es que siguen visitando la tienda.
Dado que el dataset es del año 2010 todas las recencys dan valores muy altos, se ajustara para poder analizar bien la magnitud, restandole 4500 días.

Con estos supuestos definimos los grupos como:


1.   **Leales de gran aporte**: El cluster 1 se caracteriza por tener los mejores valores de length y frequency, lo que que los define como clientes leales por largo tiempo y que suelen venir seguido a la tienda, este grupo además se caracteriza por aportar mucho a la tienda y no han parado de venir. 

2.   **Clientes nuevos de alto aporte:** A diferencia del grupo 1 el cluster 0 grupo son clientes nuevos que aportan bastante a la tienda y sería interesante retener o fidelizar.

3.    **Clientes poco leales y de bajo aporte:** EL cluster 4 son clientes antiguos que han venido pocas veces, no hacen grandes aportes y su periodicidad es la más alta del conjunto.

4.    **Clientes neutros o promedio**: Pertenecientes al cluster 5, son aquellos clientes cuyos parametros estan en el punto medio, no son tan antiguos ni nuevos, su frecuencia es buena, su gasto es promedio y visitan la tienda continuamente. 

5.  **Clientes 1 compra**:Cluster 2 son aquellos clientes que vinieron sólo una vez y no volvieron a venir, ya paso mucho tiempo entre su ultima compra y su gasto fue pequeño.

6.   **Cliente perdido**: Finalmente el cluster 3 son esos clientes que vinieron en más de una ocación, su periodicidad era buena, pero que despues de 2 o 3 compras dejaron de venir. Su aporte fue el más bajo y ya ha pasado bastante tiempo desde su ultima compra.

Esperamos que estas definiciones le sean de utilidad a MR.Lepin

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [57]:
px.scatter(x = pipe_results_x, y = pipe_results_y, color = LRMFP['Cluster'])

Como se puede ver en la gráfica a pesar de que los clusters se hayan calculado sobre todas las dimensiones de la tabla LRMFP, esta sigue manteniendo cierta distribución agrupada en su version con dimensionalidad reducida, esto puede demostrar que se siguen manteniendo las propiedades más importantes en cada cluster despues de reducir la dimensionalidad, conservaldo la lógica de la agrupacion realizada.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>